In [1]:
""" Andreas Galatis
    CS-340 Client/Server Development
    4-1 Milestone: Create and Read in Python
    August 11, 2022
    ProjectTwoDashboard.ipynb
"""

from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64
import datetime

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
import matplotlib.pyplot as plt
from bson.json_util import dumps

### COMPLETED ### : changed CRUD Python module file name and class name
from crud_animal import AnimalShelter



###########################
# Data Manipulation / Model
###########################

### COMPLETED ### : changed username and password and CRUD Python module name
username = "aacUser"
password = "Andy12"
shelter = AnimalShelter(username, password)

# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################

app = JupyterDash('Project Two Dashboard')

### COMPLETED ### : Added in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replaced image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

### COMPLETED ### : Placed HTML image tag into the app.layout code according to design
### COMPLETED ### : Added a URL anchor tag to the image
### COMPLETED ### : included name as a unique identifier 
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.A([
        html.Img(
            src='data:image/png;base64,{}'.format(encoded_image.decode()),    # image
            style = {'height' : '20%', 'width': '20%'})
    ], href='https://www.snhu.edu'),                                          # URL anchor tag 
     
    html.Center(html.B(html.H2('Dashboard by Andreas Galatis'))),             # unique identifier
    html.Hr(),
#### COMPLETED ### : Added code for interactive filtering options. 
    html.Div(className = 'row',
             style = {'display':'flex'},
             children = [
                 html.Button(id = 'submit-button-one', n_clicks_timestamp = 0, children = 'Water Rescue'),
                 html.Button(id = 'submit-button-two', n_clicks_timestamp = 0, children = 'Mountain Rescue'),
                 html.Button(id = 'submit-button-three', n_clicks_timestamp = 0, children = 'Disaster Rescue'),
                 html.Button(id = 'submit-button-four', n_clicks_timestamp = 0, children = 'Reset')
             ]),
    
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
### COMPLETED ### : Set up the features interactive data table to make it user-friendly client 
        editable = False, 
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            dcc.Graph(id='graph-id'),
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
    
])



#############################################
# Interaction Between Components / Controller
#############################################

# highlights a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]
                               

### COMPLETED ### : Added code to filter interactive data table with MongoDB queries 
@app.callback([Output('datatable-id',"data"),
               Output('datatable-id','columns')],
              [Input('submit-button-one', 'n_clicks_timestamp'),
              Input('submit-button-two', 'n_clicks_timestamp'),
              Input('submit-button-three', 'n_clicks_timestamp'),
              Input('submit-button-four', 'n_clicks_timestamp'),])

def on_click(bt1, bt2, bt3, bt4):
    
    # Initial unfiltered list of all Dog types 
    if (int(bt1) == 0 and int(bt2) == 0 and int(bt3) == 0 and int(bt4) == 0):
        df = pd.DataFrame(list(shelter.read({"animal_type": "Dog"})))
    
    # Search filter for Water rescue type
    if (int(bt1) > int(bt2) and int(bt1) > int(bt3) and int(bt1) > int(bt4)):
        df = pd.DataFrame(list(shelter.read(
            {"animal_type": "Dog",
            "breed":{"$in": [
                'Labrador Retriever Mix','Chesa Bay Retr Mix','Newfoundland Mix'
                ]},
            "sex_upon_outcome" : "Intact Female",
            "age_upon_outcome_in_weeks" : {"$gt" : 24, "$lt": 157}
            }))) 
    
    # Search filter for Mountain or Wilderness rescue type
    elif (int(bt2) > int(bt1) and int(bt2)> int(bt3) and int(bt2)> int(bt4)):
        df = pd.DataFrame(list(shelter.read(
            {"animal_type": "Dog",
            "breed":{"$in": [
                'German Shepherd','Alaskan Malamute','Old English Sheepdog',
                'Siberian Husky', 'Rottweiler'
                ]},
            "sex_upon_outcome" : "Intact Male",
            "age_upon_outcome_in_weeks" : {"$gt" : 24, "$lt": 157}
            }))) 
    
    # Search filter for Disaster or Individual Tracking rescue type
    elif (int(bt3) > int(bt1) and int(bt3)> int(bt2) and int(bt3)> int(bt4)):
        df = pd.DataFrame(list(shelter.read(
            {"animal_type": "Dog",
            "breed":{"$in": [
                'Doberman Pinscher','German Shepherd','Golden Retriever',
                'Bloodhound', 'Rottweiler'
                ]},
            "sex_upon_outcome" : "Intact Male",
            "age_upon_outcome_in_weeks" : {"$gt" : 19, "$lt": 301}
            }))) 
    
    # Option to Reset filters to all Dog types
    elif (int(bt4) > int(bt1) and int(bt4)> int(bt2) and int(bt4)> int(bt3)):
        df = pd.DataFrame(list(shelter.read({"animal_type": "Dog"})))
        
     
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
        
        
    return (data,columns)
                 
                               
# Callback for the Pie CHART                               
@app.callback(
    Output('graph-id', "figure"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ### COMPLETED #### added code for pie chart based on dog breed 
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff, names="breed")
    return fig
        
     
                               
# Callback for the MAP
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
### COMPLETED ### : Added code for geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    # Autstin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width': '1000px', 'height': '500px'}, 
               center = [30.75, -97.48],
               zoom = 10,
               children = [dl.TileLayer(id = "base-layer-id"), 
                # Marker with tool tip and popup
                        dl.Marker(position = [30.75, -97.48],
                        children = [dl.Tooltip(dff.iloc[0,4]),
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(dff.iloc[1,9])
                        ])
                ])
         ])
    ]
app